# Attendance & Risk Monitoring Project
## Data Processing & Feature Engineering

This notebook is responsible for cleaning raw data,
creating analytical tables, and applying risk classification rules.


In [181]:
import pandas as pd
from datetime import datetime, timedelta
import os

In [182]:
# 1. PATHS

BASE_PATH = r"C:\Users\jujuf\Documents\projeto_cultura_inglesa"
RAW_PATH = os.path.join(BASE_PATH, "data", "raw")
PROCESSED_PATH = os.path.join(BASE_PATH, "data", "processed")
os.makedirs(PROCESSED_PATH, exist_ok=True)

In [183]:
# 2. LOAD RAW DATAFRAMES

classes_df = pd.read_csv(os.path.join(raw_path, "classes_raw.csv"))
students_df = pd.read_csv(os.path.join(raw_path, "students_raw.csv"))
enrollments_df = pd.read_csv(os.path.join(raw_path, "enrollments_raw.csv"))
attendance_df = pd.read_csv(os.path.join(raw_path, "attendance_raw.csv"))

In [184]:
# 3. STANDARDIZE COLUMN NAMES

classes_df.rename(columns={
    'Class Code': 'class_id',
    'Branch': 'branch',
    'Teacher Full Name': 'teacher',
    'Class Shift': 'shift',
    'Course Level': 'level',
    'Branch Manager': 'branch_manager'
}, inplace=True)

students_df.rename(columns={
    'Student ID': 'student_id',
    'Student Full Name': 'name',
    'Branch': 'branch',
    'Course Level': 'level',
    'Enrollment Date': 'enrollment_date',
    'Student Status': 'status',
    'Class Code': 'class_id'
}, inplace=True)

enrollments_df.rename(columns={
    'Enrollment ID': 'enrollment_id',
    'Student ID': 'student_id',
    'Class Code': 'class_id',
    'Enrollment Status': 'status'
}, inplace=True)

attendance_df.rename(columns={
    'Enrollment ID': 'enrollment_id',
    'Class Date': 'date',
    'Attendance Flag': 'presence'
}, inplace=True)

In [185]:
# 4. DATA TYPE TREATMENT

# Classes
classes_df['class_id'] = classes_df['class_id'].astype(str)
classes_df['branch'] = classes_df['branch'].astype('category')
classes_df['teacher'] = classes_df['teacher'].astype(str)
classes_df['shift'] = classes_df['shift'].astype('category')
classes_df['level'] = classes_df['level'].astype('category')
classes_df['branch_manager'] = classes_df['branch_manager'].astype(str)

# Students
students_df['student_id'] = students_df['student_id'].astype(str)
students_df['branch'] = students_df['branch'].astype('category')
students_df['level'] = students_df['level'].astype('category')
students_df['enrollment_date'] = pd.to_datetime(students_df['enrollment_date'])
students_df['class_id'] = students_df['class_id'].astype(str)
students_df['status'] = students_df['status'].astype('category')
students_df['active'] = students_df['status'].map({'Active': True, 'Inactive': False})

# Enrollments
enrollments_df['enrollment_id'] = enrollments_df['enrollment_id'].astype(str)
enrollments_df['student_id'] = enrollments_df['student_id'].astype(str)
enrollments_df['class_id'] = enrollments_df['class_id'].astype(str)
enrollments_df['status'] = enrollments_df['status'].astype('category')
enrollments_df['active'] = enrollments_df['status'].map({'Active': True, 'Inactive': False})

# Attendance
attendance_df['enrollment_id'] = attendance_df['enrollment_id'].astype(str)
attendance_df['date'] = pd.to_datetime(attendance_df['date'])
attendance_df['presence'] = attendance_df['presence'].astype(bool)

In [186]:
# 5. CREATE ABSENCES TABLE

attendance_with_status = attendance_df.merge(
    enrollments_df[['enrollment_id', 'student_id']],
    on='enrollment_id',
    how='left'
).merge(
    students_df[['student_id', 'status']],
    on='student_id',
    how='left'
)

attendance_active = attendance_with_status[attendance_with_status['status'] == 'Active']

today = pd.Timestamp('2026-02-12')
start_period = today - pd.Timedelta(days=30)

last_30_days = attendance_active[attendance_active['date'] >= start_period]

absences_30d = last_30_days.groupby('enrollment_id')['presence'].apply(lambda x: (~x).sum()).reset_index()
absences_30d.rename(columns={'presence': 'absences_30d'}, inplace=True)

def classify_risk(absences):
    if absences >= 5:
        return 'High Risk'
    elif absences >= 3:
        return 'Medium Risk'
    else:
        return 'Normal'

absences_30d['risk'] = absences_30d['absences_30d'].apply(classify_risk)
absences_30d_df = absences_30d.copy()
absences_30d_df['enrollment_id'] = absences_30d_df['enrollment_id'].astype(str)
absences_30d_df['absences_30d'] = absences_30d_df['absences_30d'].astype(int)
absences_30d_df['risk'] = absences_30d_df['risk'].astype('category')

In [187]:
# 6. CREATE RECENT INTERVENTIONS TABLE

cord_intervention = pd.read_excel(
    os.path.join(base_path, "data", "processed", "cord_interventions.xlsx"),
    header=2
)
cord_intervention['Intervention_Date'] = pd.to_datetime(cord_intervention['Intervention_Date'])

recent_interventions = cord_intervention[
    cord_intervention['Intervention_Date'] >= start_period
][['Student_ID', 'Intervention_Type', 'Intervention_Description', 'Case_Status', 'Intervention_Date']]


C:\Users\jujuf\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [188]:
# 7. CREATE WEEKLY ABSENCES TABLE

attendance_weekly = attendance_active.copy()
attendance_weekly['year'] = attendance_weekly['date'].dt.isocalendar().year
attendance_weekly['week'] = attendance_weekly['date'].dt.isocalendar().week

weekly_absences = attendance_weekly.groupby(['enrollment_id', 'year', 'week'])['presence'] \
    .apply(lambda x: 2 - x.sum()).reset_index()
weekly_absences.rename(columns={'presence': 'absences_per_week'}, inplace=True)

def classify_weekly_risk(absences):
    if absences == 2:
        return 'Priority'
    elif absences == 1:
        return 'Attention'
    else:
        return 'Normal'

weekly_absences['risk_status'] = weekly_absences['absences_per_week'].apply(classify_weekly_risk)
weekly_absences['week_start_date'] = weekly_absences.apply(
    lambda x: datetime.strptime(f'{x.year}-W{x.week}-1', "%G-W%V-%u").date(), axis=1
)
weekly_absences['week_end_date'] = weekly_absences['week_start_date'] + timedelta(days=4)
weekly_absences['week_label'] = weekly_absences.apply(
    lambda x: f"{x.week_start_date.strftime('%d/%m/%Y')} - {x.week_end_date.strftime('%d/%m/%Y')}", axis=1
)
weekly_absences_df = weekly_absences.copy()

In [191]:
# 8. SAVE PROCESSED DATAFRAMES

students_df.to_csv(os.path.join(PROCESSED_PATH, "students.csv"), index=False, encoding="utf-8-sig")
classes_df.to_csv(os.path.join(PROCESSED_PATH, "classes.csv"), index=False, encoding="utf-8-sig")
enrollments_df.to_csv(os.path.join(PROCESSED_PATH, "enrollments.csv"), index=False, encoding="utf-8-sig")
attendance_df.to_csv(os.path.join(PROCESSED_PATH, "attendance.csv"), index=False, encoding="utf-8-sig")
absences_30d_df.to_csv(os.path.join(PROCESSED_PATH, "absences_30d.csv"), index=False, encoding="utf-8-sig")
weekly_absences_df.to_csv(os.path.join(PROCESSED_PATH, "weekly_absences.csv"), index=False, encoding="utf-8-sig")
recent_interventions.to_csv(os.path.join(PROCESSED_PATH, "recent_interventions.csv"), index=False, encoding="utf-8-sig")